In [1]:
import rootutils
rootutils.setup_root('.', indicator=".project-root", pythonpath=True)
from src.model.model_module import LitMML
from src.model.utils import get_model_and_processor
from omegaconf import OmegaConf
import torch

In [2]:
ckpt_path = '/data/bhavin/ckpts_old/93t3xgrr/ckpt-epoch=14-loss-val=2.208.ckpt' # CLIP+ITM
model, processor = get_model_and_processor(config=OmegaConf.load('configs/model/dual_encoder.yaml'))
lit_model = LitMML.load_from_checkpoint(ckpt_path, model=model, processor=processor)
model, processor = lit_model.model, lit_model.processor
itm_head = lit_model.itm_head
print(itm_head)

/home/phisch/venv_py3.8/py3.8/lib/python3.8/site-packages/lightning/pytorch/utilities/migration/utils.py:56: The loaded checkpoint was produced with Lightning v2.2.3, which is newer than your current Lightning version: v2.2.0.post0


Sequential(
  (0): Linear(in_features=1024, out_features=512, bias=True)
  (1): ReLU()
  (2): Linear(in_features=512, out_features=2, bias=True)
)


In [10]:
def _get_itm_head_predictions(itm_head, sim_f2t, top_k, features, classifier):
    batch_size = sim_f2t.shape[0]
    if sim_f2t.size(-1) < top_k:
        top_k = sim_f2t.size(-1)
    _, top_indices = torch.topk(sim_f2t, top_k, dim=-1)
    topk_class_embeds = torch.stack(
        [torch.index_select(classifier, dim=1, index=top_indices[i, :]).T for i in range(batch_size)]
    )
    multimodal_embeddings = torch.cat(
        [features.unsqueeze(1).expand(-1, top_k, -1), topk_class_embeds], dim=-1
    ).to(torch.float32)
    logits = itm_head(multimodal_embeddings)
    matching_logits = logits[:, :, 0]
    mask = torch.full_like(sim_f2t, fill_value=False, dtype=torch.bool)
    for i in range(sim_f2t.size(0)):
        mask[i, top_indices[i]] = True
    sim_f2t.fill_(-float('inf'))
    sim_f2t[mask] = matching_logits.flatten()  # Replace with new values (example)
    return sim_f2t
    # indices_max_class_logits = torch.argmax(logits[:, :, 0], dim=-1)
    # pred_classes = torch.gather(top_indices, dim=1, index=indices_max_class_logits.unsqueeze(-1))
    # return pred_classes.squeeze(-1).to(torch.int64)

In [12]:
device = "cuda"
sim_f2t = torch.randn(8, 10).to(device)
top_k = 5
features = torch.randn(8, 512).to(device)
classifier = torch.randn(512, 10).to(device)
preds = _get_itm_head_predictions(itm_head, sim_f2t, top_k, features, classifier)
print(preds)

tensor([[    -inf, -35.7969,     -inf,     -inf,     -inf,     -inf, -42.8828,
         -28.3120, -36.1812, -42.7253],
        [    -inf, -47.6826, -42.3259, -27.8318,     -inf,     -inf,     -inf,
         -53.4408, -41.4452,     -inf],
        [    -inf, -33.8399,     -inf,     -inf,     -inf, -60.0919, -42.9712,
         -60.3185,     -inf, -41.3794],
        [    -inf, -38.6740, -42.6128,     -inf,     -inf, -46.7317,     -inf,
             -inf, -45.3212, -39.5793],
        [-39.3563,     -inf, -34.7509,     -inf, -40.4056,     -inf, -38.3079,
         -24.8190,     -inf,     -inf],
        [-34.2702, -28.2009,     -inf,     -inf,     -inf, -42.9782, -52.2685,
             -inf, -51.5873,     -inf],
        [-45.7405, -46.6931,     -inf,     -inf,     -inf,     -inf,     -inf,
         -45.3044, -62.2246, -50.2090],
        [-57.9832,     -inf,     -inf,     -inf, -40.5351, -39.3979, -40.2696,
         -42.7699,     -inf,     -inf]], device='cuda:0',
       grad_fn=<IndexPutBackwa

In [22]:
a = torch.tensor([[-63.8308,  58.4294],
        [-48.5656,  45.5814],
        [-53.5590,  48.5309],
        [-19.3385,  13.9102],
        [-63.9262,  59.6700],
        [-43.9359,  36.5553]])

softmax = torch.nn.LogSoftmax(dim=1)
log_probs = torch.softmax(torch.tensor([[-1.39,  1.15]]), 1)
print(log_probs)

tensor([[0.0731, 0.9269]])
